In [ ]:
## Analytical Placement Toy ##

In [ ]:
# Step 0-1: Import Libraries
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from IPython.display import display, clear_output


In [ ]:
# Step 0-2: Define Parameters
GAMMA = 1.0
LR    = 0.5
ITER  = 40
num_bins = 16 # grid 16x16
target_density = 1.0
# NTUplace parameters
TRUNC_FACTOR = 1.0 # truncationFactor
# RePlAce parameters
DDW_TYPE = 2 # Dynamic Density Weight STRATEGY, 0: gWL/gD, 1: NTUplace, 2: RePlAce
INIT_DEN_PENALTY = 0.95 # 0.00008
HPWL_REF = 15.0
MIN_PhiCoef = 0.95
MAX_PhiCoef = 1.05

In [ ]:
class Netlist:
    def __init__(self):
        self.cells_name    = []
        self.cells_size    = []
        self.cells_pos     = [] # pos is center of cell
        self.nets          = []
        self.cells_isfixed = [] # 1: fixed, 0: movable


In [ ]:
# Step 2: Define Util Functions
# Visualize Placements
def plot_placement(ax, board_size, netlist, iteration, costs_info):
    ax.clear()
    bin_size = board_size / num_bins
    for i in range(num_bins + 1):
        x = i * bin_size
        y = i * bin_size
        ax.axhline(y, color='lightgray', linewidth=0.5, zorder=0)
        ax.axvline(x, color='lightgray', linewidth=0.5, zorder=0)

    for net in netlist.nets:
        net_cells = netlist.cells_pos[net]
        ax.plot(net_cells[:, 0], net_cells[:, 1], 'r--', alpha=0.5)  # Draw nets

    for i, (pos, size) in enumerate(zip(netlist.cells_pos, netlist.cells_size)):
        rect = plt.Rectangle(pos - size / 2, size[0], size[1], edgecolor='blue', facecolor='none')
        ax.add_patch(rect)
        ax.text(pos[0], pos[1], netlist.cells_name[i], ha='center', va='center', fontsize=8, color='blue')

    ax.set_xlim(0, board_size)
    ax.set_ylim(0, board_size)
    # ax.set_title(f'Placement at Iteration {iteration}, WL(LSE/HPWL): {costs_info["total_lse"]:.2f} / {costs_info["total_hpwl"]:.2f}, Overflow(Pot/Den): {costs_info["total_potential_overflow"]:.2f} / {costs_info["total_density_overflow"]:.2f}, Cost(obj/real): {costs_info["total_obj_value"]:.2f} / {costs_info["total_cost"]:.2f}')
    ax.set_title(f'Placement at Iteration {iteration}\n' + 
                 f'  WL: {costs_info["total_lse"]:.2f}, Overflow: {costs_info["total_potential_overflow"]:.2f},    Obj_f: {costs_info["total_obj_value"]:.2f}\n' + 
                 f'  Grad: {costs_info["gWL"]:.2e} + {costs_info["weight_density"]:.2e} * {costs_info["gPot"]:.2e} (Den Ratio: {(costs_info["weight_density"]*costs_info["gPot"]/costs_info["gWL"]):.2e})\n' + 
                 f'HPWL: {costs_info["total_hpwl"]:.2f},  OverDen: {costs_info["total_density_overflow"]:.2f}, RealCost: {costs_info["total_cost"]:.2f}')
    # ax.text(0.5, 0.92, 
    #         f'  WL: {costs_info["total_lse"]:.2f}, Overflow: {costs_info["total_potential_overflow"]:.2f},    Obj_f: {costs_info["total_obj_value"]:.2f}\n' + 
    #         f'HPWL: {costs_info["total_hpwl"]:.2f},  OverDen: {costs_info["total_density_overflow"]:.2f}, RealCost: {costs_info["total_cost"]:.2f}', 
    #      ha='center', fontsize=10)
    ax.grid(False)

# Visualize Density Map
def plot_density_map(ax, board_size, density_map):
    ax.clear()
    bin_size = board_size / num_bins
    # Draw heat map
    ax.imshow(
        density_map.T,              # transpose: make y-axis as vertical
        origin='lower',             # let (0, 0) be at the bottom-left
        extent=[0, board_size, 0, board_size],  # extend to chip size
        cmap='Reds',                # heat map
        aspect='equal'
    )
    # ax.colorbar(label='Density')
    for i in range(num_bins + 1):
        x = i * bin_size
        y = i * bin_size
        ax.axhline(y, color='lightgray', linewidth=0.5, zorder=0)
        ax.axvline(x, color='lightgray', linewidth=0.5, zorder=0)
    ax.set_xlim(0, board_size)
    ax.set_ylim(0, board_size)
    ax.set_title(f'Density Map')
    ax.grid(False)

# Initialize Placement
def initialize_placement(num_cells, board_size, isfixed):
    cells_pos = np.random.rand(num_cells, 2) * board_size  # Random positions within the canvas
    # random place movable cells at centor area (9/20 ~ 11/20)
    for i in range(num_cells):
        # rand float in [0.45, 0.55] * board_size
        if isfixed[i] == 0:
            cells_pos[i, 0] = (0.45 + 0.1 * np.random.rand()) * board_size  # x
            cells_pos[i, 1] = (0.45 + 0.1 * np.random.rand()) * board_size  # y
    return cells_pos
        

In [ ]:
# Test Case
# Assume we have 5 cells and 4 nets connecting them
board_size = 32
netlist = Netlist()
netlist.cells_name = ['A', 'B', 'C', 'D', 'E', 'F', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5', 'f_6', 'f_7', 'f_8', 'f_9', 'f_10']
# netlist.cells_size = np.array([[3,3], [3,3], [1,2], [2,1], [1,1], [1,1], [6,6], [6,6], [6,6], [6,6], [6,6], [6,6], [6,6], [6,6], [6,6], [6,6]])
netlist.cells_size = np.array([[3,3], [3,3], [1,2], [2,1], [1,1], [1,1], [3,3], [3,3], [6,6], [3,3], [3,3], [6,6], [6,6], [2,2], [2,2], [6,6]])
netlist.cells_pos = np.zeros((len(netlist.cells_name), 2))
netlist.nets = np.array([[0, 1], [1, 2], [2, 3], [3, 4], [4, 5]])
netlist.cells_isfixed = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
#nets = [[2,4,5],[0,1,3],[0,2,3,5],[0,1,2,3,4]]
num_cells = len(netlist.cells_name)

# Density Potential Initialization
bin_size = board_size / num_bins
bins_potential = np.zeros((num_bins, num_bins))  # bins current density (potential)
bins_density = np.zeros((num_bins, num_bins))  # bins current density (real, only for visualization)
bins_expect_potential = np.ones((num_bins, num_bins)) * target_density  # bins expect density (potential)
bins_free_space = np.zeros((num_bins, num_bins)) # bins initial free space (consider fixed cells)
bins_base_potential = np.zeros((num_bins, num_bins)) # bins initial potential (consider fixed cells)
cells_bins_potential = [[] for _ in range(num_cells)]  # each cell's potential to contributed bins
cells_potential_norm = np.ones(num_cells)  # each cell's potential norm
weight_density = 0.0

### LSE Wirelength Model:  
$
W_{LSE}(\mathbf{x}, \mathbf{y}) = \gamma \sum_{e \in E} \left( \ln \sum_{v_k \in e} \exp(x_k / \gamma) + \ln \sum_{v_k \in e} \exp(-x_k / \gamma)  
                                                                + \ln \sum_{v_k \in e} \exp(y_k / \gamma) + \ln \sum_{v_k \in e} \exp(-y_k / \gamma) \right)
$

In [ ]:
# Wirelength as the Log-Sum-Exp HPWL 
def calculate_lse_wl(positions, nets, gamma=1.0):
    # wl_x = r * sum( ln(sum( exp(x_k/r) )) + ln(sum( exp(-x_k/r) )) )    ### max(x) + -min(x)   ### -min(x) = max(-x)
    total_hpwl = 0.0
    for net in nets:
        xs = positions[net, 0]
        ys = positions[net, 1]
        wl_x = gamma * ( np.log(np.sum(np.exp(xs / gamma))) + np.log(np.sum(np.exp(-xs / gamma))) )
        wl_y = gamma * ( np.log(np.sum(np.exp(ys / gamma))) + np.log(np.sum(np.exp(-ys / gamma))) )
        total_hpwl += wl_x + wl_y
    return total_hpwl
def calculate_hpwl(positions, nets):
    # HPWL_x = max(x) - min(x)
    hpwl = 0
    for cell_ids in nets: # for each net
        x_coords = positions[cell_ids, 0]
        y_coords = positions[cell_ids, 1]
        hpwl += (x_coords.max() - x_coords.min()) + (y_coords.max() - y_coords.min())
    return hpwl

### Gradient Descent
We have $LSE_{x}$  
$
\small
LSE_{x} = \log \sum e^{x_i / \gamma} + \log \sum e^{-x_i / \gamma}
$  
Partial derivative with respect to $x_i$ --> we can get the gradient to optimize WL for $x_i$  
$
\small
\frac{\partial \text{LSE}_{x}}{\partial x_i} = \left( e^{x_i / \gamma} \Big/ \sum e^{x_j / \gamma} \right) - \left( e^{-x_i / \gamma} \Big/ \sum e^{-x_j / \gamma} \right)
$

In [ ]:
# Gradient descent to minimize objective function
def calculate_wl_gradient(netlist, gamma=1.0):
    cells = netlist.cells_pos; nets = netlist.nets
    grads = np.zeros_like(cells) # gradient
    for cell_ids in nets: # for each net
        xs = cells[cell_ids, 0]
        ys = cells[cell_ids, 1]
        exp_xs = np.exp(xs / gamma)       # max(x)
        exp_neg_xs = np.exp(-xs / gamma)  # -min(x)
        exp_ys = np.exp(ys / gamma)       # max(y)
        exp_neg_ys = np.exp(-ys / gamma)  # -min(y)

        grad_xs = gamma * ( (exp_xs / np.sum(exp_xs)) - (exp_neg_xs / np.sum(exp_neg_xs)) ) # partial derivative for each x_i
        grad_ys = gamma * ( (exp_ys / np.sum(exp_ys)) - (exp_neg_ys / np.sum(exp_neg_ys)) )

        grads[cell_ids, 0] += grad_xs
        grads[cell_ids, 1] += grad_ys
    return grads

### Bell-Shaped Potential Density Model:  
$
p_x(b, v) = 
\begin{cases} 
1 - a d_x^2, & 0 \leq d_x \leq \frac{w_v}{2} + w_b \\
b \left(d_x - \frac{w_v}{2} - 2w_b \right)^2, & \frac{w_v}{2} + w_b \leq d_x \leq \frac{w_v}{2} + 2w_b \\
0, & \frac{w_v}{2} + 2w_b \leq d_x 
\end{cases}
\quad
\text{, where} \quad
a = \frac{4}{(w_v + 2w_b)(w_v + 4w_b)}, \quad
b = \frac{2}{w_b(w_v + 4w_b)}
$  

### Gradient Descent  
Partial derivative with respect to $x_i$ --> we can get the gradient to optimize Density for $x_i$  
$
\frac{\partial p_x(b, v)}{\partial x_v} = 
\begin{cases}
-2ad_x, & 0 \leq d_x \leq \frac{w_v}{2} + w_b \\
2b \left(d_x - \frac{w_v}{2} - 2w_b \right), & \frac{w_v}{2} + w_b \leq d_x \leq \frac{w_v}{2} + 2w_b \\
0, & \frac{w_v}{2} + 2w_b \leq d_x
\end{cases}
\quad \text{with} \quad
d_x = |x_v - x_b|
$

In [ ]:
# Bell-Shaped Potential 
# Refer to Eq. (6,7) and Fig. 2(b) in the paper
# potential_x(b, v)
def get_potential(xv, xb, cell_size, bin_size):
    # Return the potential value of the overlap area (1-dir overlapping for one cell to one bin)
    d = abs(xv - xb)
    w = cell_size
    wb = bin_size
    if d <= w/2 + wb:
        a = 4 / ((w + 2*wb)*(w + 4*wb))
        return 1 - a * d**2
    elif d <= w/2 + 2*wb:
        b = 2 / (wb*(w + 4*wb))
        return b * (d - w/2 - 2*wb)**2
    else:
        return 0.0
# partial derivative of potential_x(b, v) w.r.t. xv
def get_potential_gradient(xv, xb, cell_size, bin_size):
    d = abs(xv - xb)
    w = cell_size
    wb = bin_size
    if xv >= xb: # right half
        if d <= w/2 + wb:
            a = 4 / ((w + 2*wb)*(w + 4*wb))
            return -2 * a * d 
        elif d <= w/2 + 2*wb:
            b = 2 / (wb*(w + 4*wb))
            return 2 * b * (d - w/2 - 2*wb)
        else:
            return 0.0
    else: # left half
        if d <= w/2 + wb:
            a = 4 / ((w + 2*wb)*(w + 4*wb))
            return 2 * a * d
        elif d <= w/2 + 2*wb:
            b = 2 / (wb*(w + 4*wb))
            return -2 * b * (d - w/2 - 2*wb)
        else:
            return 0.0
def get_overlap(xv, xb, cell_size, bin_size):
    # Return the overlap dist of one cell to one bin
    x1 = xv - cell_size / 2
    x2 = xv + cell_size / 2
    b1 = xb - bin_size / 2
    b2 = xb + bin_size / 2
    if x1 > b2 or x2 < b1:
        return 0.0
    else:
        return min(x2, b2) - max(x1, b1)
    

In [ ]:
def find_contribute_bins(x, y, w, h):
    # Find contributed bins (influence bin range < 2 bins around cell, bin_x = cell_cx +- cell_w/2 + 2*num_bins)
    min_bin_x = max(0, int((x - (w/2 + 2*bin_size)) // bin_size))
    max_bin_x = min(num_bins-1, int((x + (w/2 + 2*bin_size)) // bin_size))
    min_bin_y = max(0, int((y - (h/2 + 2*bin_size)) // bin_size))
    max_bin_y = min(num_bins-1, int((y + (h/2 + 2*bin_size)) // bin_size))
    return min_bin_x, max_bin_x, min_bin_y, max_bin_y

In [ ]:
# Update Potential for fixed cells
def update_potential_base(): # UpdatePotentialGridBase()
    global bins_free_space, bins_base_potential
    bins_base_potential = np.zeros((num_bins, num_bins)) # initial potential = 0.0
    bins_free_space = np.full((num_bins, num_bins), bin_size*bin_size) # initial free space = bin_size*bin_size (empty bin)

    for idx, cell in enumerate(netlist.cells_pos):
        if netlist.cells_isfixed[idx]:  # If the cell is fixed
            x, y = cell
            w, h = netlist.cells_size[idx]
            min_bin_x, max_bin_x, min_bin_y, max_bin_y = find_contribute_bins(x, y, w, h)
            # bx, by are bin index (not position)
            for bx in range(min_bin_x, max_bin_x+1):
                for by in range(min_bin_y, max_bin_y+1):
                    bin_cx = bx * bin_size + bin_size/2
                    bin_cy = by * bin_size + bin_size/2
                    potential_x = get_potential(x, bin_cx, w, bin_size)
                    potential_y = get_potential(y, bin_cy, h, bin_size)
                    potential = potential_x * potential_y * w * h 
                    overlap_x = get_overlap(x, bin_cx, w, bin_size)
                    overlap_y = get_overlap(y, bin_cy, w, bin_size)
                    overlap = overlap_x * overlap_y
                    bins_free_space[bx, by] -= overlap
                    bins_base_potential[bx, by] += potential
    return

def level_smooth_base_potential(basePotential, delta): # LevelSmoothBasePotential
    oldPotential = basePotential.copy()
    maxPotential = np.max(oldPotential)
    totalPotential = np.sum(oldPotential)
    avgPotential = totalPotential / oldPotential.size
    if totalPotential == 0:
        return  # 無 preplaced block
    # Apply TSP-style smoothing
    newPotential = np.zeros_like(oldPotential)
    for i in range(basePotential.shape[0]):
        for j in range(basePotential.shape[1]):
            val = oldPotential[i][j]
            if val >= avgPotential:
                newPotential[i][j] = avgPotential + ((val - avgPotential) / maxPotential) ** delta * maxPotential
            else:
                newPotential[i][j] = avgPotential - ((avgPotential - val) / maxPotential) ** delta * maxPotential
    # Normalize total potential to match original
    newTotal = np.sum(newPotential)
    ratio = totalPotential / newTotal if newTotal != 0 else 1.0
    basePotential[:, :] = newPotential * ratio
    
def smooth_base_potential(smooth_r=5, smooth_delta=1.0): # SmoothBasePotential()
    global bins_base_potential
    # Step 1: Gaussian smoothing
    smoothed = gaussian_filter(bins_base_potential, sigma=smooth_r)
    bins_base_potential[:, :] = smoothed
    # Step 2: Level smoothing (only for multi-level, smooth_delta > 1)
    if smooth_delta > 1.001:
        level_smooth_base_potential(bins_base_potential, smooth_delta) # pass by reference
        return
    # Step 3: Additional height boost for fully blocked bins (Find the bins that are fully blocked but being too much smoothed by Gaussian)
    more_smooth = gaussian_filter(bins_base_potential, sigma=smooth_r * 6) # more smoothed version for boost
    bin_area = bin_size * bin_size
    half_bin_area = bin_area / 2
    scale = 3
    for i in range(bins_base_potential.shape[0]):
        for j in range(bins_base_potential.shape[1]):
            free = bin_area - bins_base_potential[i][j]
            # If the bin's potential is already no space or high enough, boost its potential
            if free < 1e-4 and more_smooth[i][j] > half_bin_area:
                bins_base_potential[i][j] += (more_smooth[i][j] - half_bin_area) * scale

def update_exp_bin_potential(target_util=1.0): # UpdateExpBinPotential
    global bins_base_potential, bins_expect_potential
    bins_expect_potential = np.zeros_like(bins_base_potential)
    total_free = 0
    zero_bin = 0
    for i in range(num_bins):
        for j in range(num_bins):
            base = bins_base_potential[i, j]
            # Calculate the free space available in the bin
            free = bin_size * bin_size - base  # Simplified for illustration
            if( free > 1e-4 ):
                bins_expect_potential[i, j] = free * target_util
                total_free += bins_expect_potential[i, j]
            else:
                zero_bin += 1
    
    totalMovableModuleArea = np.sum(netlist.cells_size[netlist.cells_isfixed == 0][:, 0] * netlist.cells_size[netlist.cells_isfixed == 0][:, 1])
    algUtil = totalMovableModuleArea / total_free
    print(f"PBIN: Zero space bin #= {zero_bin} ({100*zero_bin/num_bins/num_bins}%).  Algorithm utilization= {algUtil}")
# Init potential
def init_density_model():
    global bins_potential, bins_density, bins_expect_potential, bins_free_space, bins_base_potential, cells_bins_potential, cells_potential_norm, weight_density, MIN_PhiCoef, MAX_PhiCoef
    bins_potential = np.zeros((num_bins, num_bins))  # bins current density (potential)
    bins_density = np.zeros((num_bins, num_bins))  # bins current density (real, only for visualization)
    bins_expect_potential = np.ones((num_bins, num_bins)) * target_density  # bins expect density (potential)
    bins_free_space = np.zeros((num_bins, num_bins)) # bins initial free space (consider fixed cells)
    bins_base_potential = np.zeros((num_bins, num_bins)) # bins initial potential (consider fixed cells)
    cells_bins_potential = [[] for _ in range(num_cells)]  # each cell's potential to contributed bins
    cells_potential_norm = np.ones(num_cells)  # each cell's potential norm
    weight_density = 0.0
    MIN_PhiCoef = 0.95
    MAX_PhiCoef = 1.05

In [ ]:
# Compute New Potential for Bins
def compute_new_potential_grid():
    global cells_bins_potential, cells_potential_norm
    cells_bins_potential = [[] for _ in range(num_cells)]
    cells_potential_norm = np.ones(num_cells)
    for i, (x, y) in enumerate(netlist.cells_pos):
        if netlist.cells_isfixed[i]:  # If the cell is fixed
            continue
        w, h = netlist.cells_size[i]
        total_potential = 0
        # Find contributed bins (influence bin range < 2 bins around cell, bin_x = cell_cx +- cell_w/2 + 2*num_bins)
        min_bin_x, max_bin_x, min_bin_y, max_bin_y = find_contribute_bins(x, y, w, h)
        # bx, by are bin index (not position)
        for bx in range(min_bin_x, max_bin_x+1):
            for by in range(min_bin_y, max_bin_y+1):
                bin_cx = bx * bin_size + bin_size/2
                bin_cy = by * bin_size + bin_size/2
                potential_x = get_potential(x, bin_cx, w, bin_size)
                potential_y = get_potential(y, bin_cy, h, bin_size)
                potential = potential_x * potential_y * w * h  # 乘上 cell 面積
                if potential > 0:
                    cells_bins_potential[i].append((bx, by, potential))
                    total_potential += potential
        cells_potential_norm[i] = netlist.cells_size[i][0] * netlist.cells_size[i][1] / total_potential 

In [ ]:
# Update Potential for Bins
def update_potential_grid():
    global bins_potential
    bins_potential[:, :] = 0 # np.zeros((num_bins, num_bins))
    for i in range(num_cells):
        for bx, by, pot in cells_bins_potential[i]:
            bins_potential[bx, by] += pot * cells_potential_norm[i]
    # only for visualization, check golden density
    global bins_density
    bins_density[:, :] = 0
    bins_used_area = np.zeros((num_bins, num_bins))
    for i in range(num_cells):
        for bx, by, pot in cells_bins_potential[i]:
            x1, x2, y1, y2 = netlist.cells_pos[i][0] - netlist.cells_size[i][0]/2, netlist.cells_pos[i][0] + netlist.cells_size[i][0]/2, netlist.cells_pos[i][1] - netlist.cells_size[i][1]/2, netlist.cells_pos[i][1] + netlist.cells_size[i][1]/2
            bx1, bx2, by1, by2 = bx * bin_size, (bx+1) * bin_size, by * bin_size, (by+1) * bin_size
            overlap_area = max(0, min(x2, bx2) - max(x1, bx1)) * max(0, min(y2, by2) - max(y1, by1))
            bins_used_area[bx, by] += overlap_area
    bins_density = bins_used_area / (bin_size**2)


In [ ]:
# Get Potential Gradient
def calculate_density_gradient():
    cells = netlist.cells_pos
    grads = np.zeros_like(cells) # gradient
    for i in range(num_cells):
        grad_x = grad_y = 0
        for bx, by, pot in cells_bins_potential[i]:
            bin_cx = bx * bin_size + bin_size/2
            bin_cy = by * bin_size + bin_size/2
            gx = get_potential_gradient(cells[i, 0], bin_cx, netlist.cells_size[i][0], bin_size)
            gy = get_potential_gradient(cells[i, 1], bin_cy, netlist.cells_size[i][1], bin_size)
            diff = bins_potential[bx, by] - bins_expect_potential[bx, by] # loss (distance between target)

            grad_x += gx * diff
            grad_y += gy * diff
        grads[i, 0] = grad_x
        grads[i, 1] = grad_y
    return grads

In [ ]:
# calculate overflow potential
def calculate_overflow_potential():
    overflow = np.maximum(bins_potential - bins_expect_potential, 0)
    return np.sum(overflow), overflow, np.max(overflow)
def calculate_overflow_density():
    overflow = np.maximum(bins_density - target_density, 0)
    return np.sum(overflow), overflow, np.max(overflow)

### Objective Function  
$
\min \; W(\mathbf{x}, \mathbf{y}) + 0.5\lambda \sum_b \left( \hat{D}_b(\mathbf{x}, \mathbf{y}) - M_b \right)^2
$

In [ ]:
def adjust_force(gradients):
    # grad truncation with expMaxGradSquare=(avgGrad * param.truncationFactor)^2
    cells = netlist.cells_pos
    totalGrad = 0
    for i in range(num_cells):
        totalGrad += gradients[i, 0] * gradients[i, 0] + gradients[i, 1] * gradients[i, 1]
    avgGrad = math.sqrt( totalGrad / num_cells )
    expMaxGrad = avgGrad * TRUNC_FACTOR # x + y
    expMaxGradSquare = expMaxGrad * expMaxGrad

    for i in range(num_cells):
        gradSquare = gradients[i, 0] * gradients[i, 0] + gradients[i, 1] * gradients[i, 1]
        if gradSquare > expMaxGradSquare:
            scale = expMaxGrad / math.sqrt(gradSquare)
            gradients[i, 0] *= scale
            gradients[i, 1] *= scale
    return gradients
def get_phi_coef(scaled_diff_hpwl):
    new_coef = MAX_PhiCoef if (scaled_diff_hpwl < 0) else MAX_PhiCoef * pow( MAX_PhiCoef, scaled_diff_hpwl * -1.0 )
    new_coef = max(MIN_PhiCoef, new_coef)
    return new_coef

def calculate_objective_gradient(lambda_prev=None, hpwl=0, hpwl_prev=0):
    cells = netlist.cells_pos
    grads = np.zeros_like(cells) # gradient
    grad_pot = calculate_density_gradient() # get potential gradients
    grad_wl = calculate_wl_gradient(netlist) # get wirelength gradients
    # Determine the new density_weight (lambda)
    weight_density = 0.0
    if DDW_TYPE == 0: # Just use gWL/gD
        weight_density = np.sum(np.abs(grad_wl)) / np.sum(np.abs(grad_pot))
    elif DDW_TYPE == 1: # NTUplace: density_weight (lambda) is initialized according to the strength of wirelength and density gradients, and is increased by two times for each iteration
        if lambda_prev is None:
            weight_density = np.sum(np.abs(grad_wl)) / np.sum(np.abs(grad_pot))
        else:
            weight_density = lambda_prev * 2
    elif DDW_TYPE == 2: # RePlAce:
        if lambda_prev is None:
            weight_density = np.sum(np.abs(grad_wl)) / np.sum(np.abs(grad_pot)) * INIT_DEN_PENALTY
        else:
            phi_coef = get_phi_coef( (hpwl - hpwl_prev) / HPWL_REF )
            weight_density = lambda_prev * phi_coef
    grads = grad_wl + 0.5 * grad_pot * weight_density  # combine gradients

    # AdjustForce(): truncation grad with expMaxGradSquare=(avgGrad * param.truncationFactor)^2
    grads = adjust_force(grads)

    return grads, weight_density, grad_wl, grad_pot

In [ ]:
# calculate objective value and real cost
def calculate_obj_value(total_wl, total_potential, _lambda):
    obj_value = total_wl + 0.5 * _lambda * total_potential
    return obj_value
def calculate_cost(total_hpwl, total_density, _lambda):
    cost = total_hpwl + 0.5 * _lambda * total_density
    return cost

### Update cell positions using gradient descent  
$
x_k = x_{k-1} - stepSize * gradient
$

In [ ]:
# Update cell positions using gradient descent
def update_positions(cells_pos, gradient, learning_rate=0.1, is_fixed=None):
    if is_fixed is None:
        is_fixed = np.zeros(len(cells_pos), dtype=bool)
    is_fixed = is_fixed.astype(bool)
    movable_mask = ~is_fixed[:, None]  # shape: (N, 1)
    cells_pos -= learning_rate * gradient * movable_mask
    return cells_pos

# Ensure cells stay within the board boundaries
def bound_cells(cells_pos):
    # pos = boundary +- cell_size / 2
    cells_pos[:, 0] = np.clip(cells_pos[:, 0], netlist.cells_size[:, 0] / 2, board_size - netlist.cells_size[:, 0] / 2)
    cells_pos[:, 1] = np.clip(cells_pos[:, 1], netlist.cells_size[:, 1] / 2, board_size - netlist.cells_size[:, 1] / 2)
    return cells_pos

### Initial place

In [ ]:
# Initialize random positions for cells (2D positions: x, y)
np.random.seed(666)  # For reproducibility
init_pos = initialize_placement(len(netlist.cells_name), board_size, netlist.cells_isfixed)
init_pos = bound_cells(init_pos)
# Initialize bins expected potential
init_density_model()
update_potential_base()
smooth_base_potential()
update_exp_bin_potential()

### Main analytical placement function ## 

In [ ]:
## Main analytical placement function ## 
# Initialize cells positions and parameters
netlist.cells_pos = init_pos.copy()
num_iter = ITER
gamma = GAMMA
step_size = LR
weight_density = None
DDW_TYPE = 0 # Dynamic Density Weight STRATEGY, 0: gWL/gD, 1: NTUplace, 2: RePlAce

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

early_stop = False
enough_iter = 15
total_over_den_prev = 0
max_den_prev = 0
# Iteratively optimize placement
for iter in range(num_iter):
    # Calculate wirelength
    total_hpwl = calculate_hpwl(netlist.cells_pos, netlist.nets)
    total_lse = calculate_lse_wl(netlist.cells_pos, netlist.nets, gamma=gamma)
    # Calculate Density
    compute_new_potential_grid()
    update_potential_grid()
    total_density_overflow, density_overflow_map, max_den = calculate_overflow_density()
    total_potential_overflow, potential_overflow_map, max_pot = calculate_overflow_potential()

    # Calculate gradient
    gradient, lambda_new, gWL, gPot = calculate_objective_gradient(lambda_prev=weight_density) # actually here is calculate_potentail_gradient
    weight_density = lambda_new
    # Update positions
    netlist.cells_pos = update_positions(netlist.cells_pos, gradient, learning_rate=step_size, is_fixed=netlist.cells_isfixed)
    netlist.cells_pos = bound_cells(netlist.cells_pos) # bound cells to board size
    # Calculate objective value
    total_obj_value = calculate_obj_value(total_lse, total_potential_overflow, weight_density)
    total_cost = calculate_cost(total_hpwl, total_density_overflow, weight_density)

    # Early stop
    enoughIter = iter > enough_iter
    spreadEnough = total_density_overflow < target_density + 0.2
    increaseOverDen = total_density_overflow > total_over_den_prev
    increaseMaxDen = max_den > max_den_prev
    notEfficientOptimize = 0.5 * total_potential_overflow * weight_density / total_obj_value * 100.0 > 95
    # if enoughIter and np.linalg.norm(gradient) > np.linalg.norm(g_prev):
    #     # print(f'Converged at iteration {iter}')
    #     early_stop = True
    if enoughIter and notEfficientOptimize:
        print(f'Failed to further optimize at iteration {iter}')
        early_stop = True
    if enoughIter and spreadEnough and increaseOverDen and increaseMaxDen:
        print(f'Cannot further reduce over density at iteration {iter}')
        early_stop = True
    # update last values
    total_over_den_prev = total_density_overflow
    max_den_prev = max_den


    # Plot placement
    costs_info = {"total_hpwl": total_hpwl, "total_lse": total_lse, "total_density_overflow": total_density_overflow, "total_potential_overflow": total_potential_overflow, "total_obj_value": total_obj_value, "total_cost": total_cost}
    plot_placement(axs[0], board_size, netlist, iter, costs_info)
    plot_density_map(axs[1], board_size, bins_density)
    display(fig)
    clear_output(wait=True)
    plt.pause(0.01)

    if early_stop:
        break

### Better Gradient Descent: CG

In [ ]:
# CG (conjugate gradient) with Dynemic Step Size
# Update cell positions
def update_positions_cg(cells_pos, g_prev, d_prev, grad, step_size=1.0, is_fixed=None):
    # (0) set movable mask
    if is_fixed is None:
        is_fixed = np.zeros(len(cells_pos), dtype=bool)
    is_fixed = is_fixed.astype(bool)
    movable_mask = ~is_fixed[:, None]  # shape: (N, 1)

    # (1) We have gradient directions grad = g_k = ∇f(x_k)
    # (2) Compute Polak-Ribiere parameter β_k (make history grad have bigger ratio if gradient differ too much)
    beta = np.sum(grad * (grad - g_prev), axis=1) / (np.sum(g_prev**2, axis=1) + 1e-10)
    beta = np.clip(beta, 0, None)  # make sure β_k >= 0
    # (3) Compute conjugate directions d = -grad + beta*d_prev
    dir = -grad + beta[:, None] * d_prev # d_prev.shape = (N, 2)
    # (4) Compute step size alpha = s/norm(d)
    alpha = step_size / (np.linalg.norm(dir, axis=1) + 1e-10) # small d --> big step, big d --> small step (improve convergence)
    # (5) update positions: x = x_prev + alpha*d
    cells_pos += (alpha[:, None] * dir) * movable_mask
    return cells_pos, dir

In [ ]:
## Main analytical placement function ## 
# Initialize cells positions and parameters
netlist.cells_pos = init_pos.copy()
num_iter = ITER
gamma = GAMMA
step_size = board_size / 32
weight_density = None
DDW_TYPE = 2 # Dynamic Density Weight STRATEGY, 0: gWL/gD, 1: NTUplace, 2: RePlAce

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

g_prev = np.zeros_like(netlist.cells_pos)  # init grad = 0
d_prev = np.zeros_like(netlist.cells_pos)  # init dir = 0
hpwl_prev = 0
early_stop = False
enough_iter = 15
total_over_den_prev = 0
max_den_prev = 0
max_backtrak = 1 # 10
isMaxPhiCoefChanged = False
# Iteratively optimize placement
for iter in range(num_iter):
    # Calculate wirelength
    total_hpwl = calculate_hpwl(netlist.cells_pos, netlist.nets)
    total_lse = calculate_lse_wl(netlist.cells_pos, netlist.nets, gamma=gamma)
    # Calculate Density
    compute_new_potential_grid()
    update_potential_grid()
    total_density_overflow, density_overflow_map, max_den = calculate_overflow_density()
    total_potential_overflow, potential_overflow_map, max_pot = calculate_overflow_potential()

    # Calculate gradient
    gradient, lambda_new, gWL, gPot = calculate_objective_gradient(lambda_prev=weight_density, hpwl=total_hpwl, hpwl_prev=hpwl_prev) # actually here is calculate_potentail_gradient
    weight_density = lambda_new
    # Update positions
    netlist.cells_pos, dk = update_positions_cg(netlist.cells_pos, g_prev, d_prev, gradient, step_size=step_size, is_fixed=netlist.cells_isfixed)
    netlist.cells_pos = bound_cells(netlist.cells_pos) # bound cells to board size
    # Calculate objective value
    total_obj_value = calculate_obj_value(total_lse, total_potential_overflow, weight_density)
    total_cost = calculate_cost(total_hpwl, total_density_overflow, weight_density)

    # Check early stop
    enoughIter = iter > enough_iter
    spreadEnough = total_density_overflow < target_density + 0.2
    increaseOverDen = total_potential_overflow > total_over_den_prev # total_density_overflow > total_over_den_prev
    increaseMaxDen = max_den > max_den_prev
    notEfficientOptimize = 0.5 * total_potential_overflow * weight_density / total_obj_value * 100.0 > 95
    if enoughIter and notEfficientOptimize:
        print(f'Failed to further optimize at iteration {iter}')
        early_stop = True
    if enoughIter and spreadEnough and increaseOverDen and increaseMaxDen:
        print(f'Cannot further reduce over density at iteration {iter}')
        early_stop = True
    # update last values
    g_prev = gradient.copy(); d_prev = dk.copy()
    total_over_den_prev = total_potential_overflow #total_density_overflow
    max_den_prev = max_den
    hpwl_prev = total_hpwl
    if isMaxPhiCoefChanged and ((iter) % max_backtrak) == 0:
        isMaxPhiCoefChanged = False
    if isMaxPhiCoefChanged and ((iter+1) % max_backtrak) == 0:
        isMaxPhiCoefChanged = True
        MAX_PhiCoef *= 0.99
    

    # Plot placement
    costs_info = {"total_hpwl": total_hpwl, "total_lse": total_lse, "total_density_overflow": total_density_overflow, "total_potential_overflow": total_potential_overflow, "total_obj_value": total_obj_value, "total_cost": total_cost, "weight_density": weight_density, "gWL": np.sum(np.abs(gWL)), "gPot": np.sum(np.abs(gPot))}
    plot_placement(axs[0], board_size, netlist, iter, costs_info)
    plot_density_map(axs[1], board_size, bins_potential) # plot_density_map(axs[1], board_size, bins_density)
    display(fig)
    clear_output(wait=True)
    plt.pause(0.001)

    if early_stop:
        break

In [ ]:
from datetime import datetime
import os
## Write bookshelf
def write_aux(design_name, folder):
    with open(f'{folder}/{design_name}.aux', 'w') as fp:
        fp.write(f"RowBasedPlacement :  {design_name}.nodes  {design_name}.nets  {design_name}.wts  {design_name}.pl  {design_name}.scl\n")
def write_nodes(design_name, folder, date=None):
    with open(f'{folder}/{design_name}.nodes', 'w') as fp:
        fp.write("UCLA nodes 1.0\n")
        fp.write(f"# Created      : {date}\n")
        fp.write("# User         : frankchiang@eda.ee.ntu.edu.tw (Cheng-Yu Chiang)\n")
        fp.write("# Institution  : EDA Lab, NTU\n\n")
        fp.write(f"NumNodes : {len(netlist.cells_name)}\n")
        fp.write(f"NumTerminals : {np.sum(netlist.cells_isfixed)}\n")
        for i in range(len(netlist.cells_name)):
            fp.write(f"{netlist.cells_name[i]}    {netlist.cells_size[i][0]}    {netlist.cells_size[i][1]}" + ("    terminal" if netlist.cells_isfixed[i]==1 else "") + "\n")
def write_pl(design_name, folder, date=None):
    with open(f'{folder}/{design_name}.pl', 'w') as fp:
        fp.write("UCLA pl 1.0\n")
        fp.write(f"# Created      : {date}\n")
        fp.write("# User         : frankchiang@eda.ee.ntu.edu.tw (Cheng-Yu Chiang)\n")
        fp.write("# Institution  : EDA Lab, NTU\n\n")
        for i in range(len(netlist.cells_name)):
            fp.write(f"{netlist.cells_name[i]}    {int(init_pos[i][0]-netlist.cells_size[i][0]/2)}    {int(init_pos[i][1]-netlist.cells_size[i][1]/2)}" + ("    : N" if netlist.cells_isfixed[i]==0 else "    : N /FIXED") + "\n")
def write_nets(design_name, folder, date=None):
    with open(f'{folder}/{design_name}.nets', 'w') as fp:
        fp.write("UCLA pl 1.0\n")
        fp.write(f"# Created      : {date}\n")
        fp.write("# User         : frankchiang@eda.ee.ntu.edu.tw (Cheng-Yu Chiang)\n")
        fp.write("# Institution  : EDA Lab, NTU\n\n")
        fp.write(f"NumNets : {len(netlist.nets)}\n")
        fp.write(f"NumPins : {sum(len(net) for net in netlist.nets)}\n")
        for i in range(len(netlist.nets)):
            fp.write(f"NetDegree : {len(netlist.nets[i])}\n")
            for j in range(len(netlist.nets[i])):
                fp.write(f"    {netlist.cells_name[netlist.nets[i][j]]} I : 0    0\n")
def write_wts(design_name, folder, date=None):
    with open(f'{folder}/{design_name}.wts', 'w') as fp:
        fp.write("UCLA pl 1.0\n")
        fp.write(f"# Created      : {date}\n")
        fp.write("# User         : frankchiang@eda.ee.ntu.edu.tw (Cheng-Yu Chiang)\n")
        fp.write("# Institution  : EDA Lab, NTU\n\n")
def write_scl(design_name, folder, date=None):
    with open(f'{folder}/{design_name}.scl', 'w') as fp:
        fp.write("UCLA pl 1.0\n")
        fp.write(f"# Created      : {date}\n")
        fp.write("# User         : frankchiang@eda.ee.ntu.edu.tw (Cheng-Yu Chiang)\n")
        fp.write("# Institution  : EDA Lab, NTU\n\n")
        fp.write(f"NumRows : {board_size}\n")
        fp.write("\n")
        for i in range(board_size):
            fp.write(f"CoreRow Horizontal\n")
            fp.write(f"  Coordinate    :   {i}\n")
            fp.write(f"  Height        :   {1}\n")
            fp.write(f"  Sitewidth     :   {1}\n")
            fp.write(f"  Sitespacing   :   {1}\n")
            fp.write(f"  Siteorient    :   {0}\n")
            fp.write(f"  Sitesymmetry  :   {1}\n")
            fp.write(f"  SubrowOrigin  :   {0}    NumSites  :  {board_size}\n")
            fp.write(f"END\n")
def write_bookshelf(design_name, folder, date=None):
    os.system(f"rm -rf {folder} && mkdir -p {folder}")
    write_aux(design_name, folder)
    write_nodes(design_name, folder, date=date)
    write_pl(design_name, folder, date=date)
    write_nets(design_name, folder, date=date)
    write_wts(design_name, folder, date=date)
    write_scl(design_name, folder, date=date)
    print(f"Write {folder}/{design_name}.aux, {design_name}.nodes, {design_name}.pl, {design_name}.nets, {design_name}.wts, {design_name}.scl")

write_bookshelf("toy0", "../../benchmarks/toy", date=datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S"))